# Runs CNMFE on motion corrected videos

In [ ]:
from datetime import datetime
import scipy.io as sio
import re
import os
import h5py
import csv
import tensorflow as tf
import time
import logging
import zipfile
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('default')
import numpy as np
from moviepy.editor import *
import smtplib

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
import peakutils

import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')

In [ ]:
#%% restart cluster to clean up memory
#if 'dview' in locals():
#    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=2, single_thread=False, ignore_preexisting=True)
n_processes

## Load Motion Corrected data

In [ ]:
import pandas as pd
params_csv = pd.read_csv('/home/prez/neurodata/cheeseboard-down/down_2/2020-10/cnmfe_params.csv')
animal_params = params_csv[params_csv['animal'] == 'O-TR']

In [ ]:
src_dir1 = '/home/prez/neurodata/cheeseboard-down/down_2/2020-10/learning/2020_10_22/trial/O-TR/Session1/13_00_16/Miniscope/'
fnames = ['memmap_0000_d1_112_d2_174_d3_1_order_C_frames_1000_.mmap',
          'memmap_0001_d1_112_d2_174_d3_1_order_C_frames_433_.mmap']
fpaths = [src_dir1 + f for f in fnames]
src_dir2 = '/home/prez/neurodata/cheeseboard-down/down_2/2020-10/learning/2020_10_22/trial/O-TR/Session8/13_25_55/Miniscope/'
fnames = ['memmap_0011_d1_112_d2_174_d3_1_order_C_frames_911_.mmap',
          'memmap_0012_d1_112_d2_174_d3_1_order_C_frames_169_.mmap']
          
fpaths = fpaths + [src_dir2 + f for f in fnames]

input_mmap_path = fpaths[0]
if len(fpaths) > 0:
    input_mmap_path = cm.save_memmap(fpaths, base_name='memmap_', order='C',border_to_0=0)

# load memory mappable file
Yr, dims, T = cm.load_memmap(input_mmap_path)
images = Yr.T.reshape((T,) + dims, order='F')
images.shape

In [ ]:
# Compute some summary images (correlation and peak to noise) while downsampling temporally 5x to speedup the process and avoid memory overflow
cn_filter, pnr = cm.summary_images.correlation_pnr(images[::5], gSig=3, swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile

#Plot the results of the correlation/PNR projection
#plt.figure(figsize=(20,10))
#plt.subplot(2, 2, 1); plt.imshow(cn_filter); plt.colorbar(); plt.title('Correlation projection')
#plt.subplot(2, 2, 2); plt.imshow(pnr); plt.colorbar(); plt.title('PNR')

inspect_correlation_pnr(cn_filter, pnr)             

## Run CNMFE

In [ ]:
gSig = (4, 4)
gSiz = (19, 19)  # average diameter of a neuron, in general 4*gSig+1
decay_time_sec = 0.4
min_corr = .8  # min peak value from correlation image
min_pnr = 8  # min peak to noise ration from PNR image
ring_size_factor = 1.6  # radius of ring is gSiz*ring_size_factor
    
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None for 1p data
merge_thr = .7      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 2            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 0             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
ssub_B = 2          # additional downsampling factor in space for background


opts = params.CNMFParams(params_dict={'dims': dims,
                                'fr': 20,
                                'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'decay_time': decay_time_sec,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': 2})                # number of pixels to not consider in the borders)


In [ ]:
start = time.time()
# Perform CNMF
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=None, params=opts)
cnm.fit(images)

end = time.time()
print(end - start)

## Component Evaluation
The components are evaluated in three ways:
- the shape of each component must be correlated with the data
- a minimum peak SNR is required over the length of a transient
- each shape passes a CNN based classifier

In [ ]:
#%% COMPONENT EVALUATION

#import os
#os.environ["CAIMAN_DATA"] = '/mnt/DATA/Prez/code/miniscope_repos/CaImAn'
min_SNR = 2.5         # adaptive way to set threshold on the transient size
r_values_min = 0.8    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': True,
                           'min_cnn_thr': 0.99,
                           'cnn_lowest': 0.1
                          })
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

In [ ]:
#plt.hist(cnm.estimates.cnn_preds)
cnm.estimates.cnn_preds

## Plot results

In [ ]:
neuronsToPlot = 20

DeconvTraces = cnm.estimates.S[cnm.estimates.idx_components,:]
RawTraces = cnm.estimates.C[cnm.estimates.idx_components,:]
SFP = cnm.estimates.A[:,cnm.estimates.idx_components]
SFP_dims = list(dims)
SFP_dims.append(SFP.shape[1])
print('Spatial foootprints dimensions (height x width x neurons): ' + str(SFP_dims))

numNeurons = SFP_dims[2]

SFP = np.reshape(SFP.toarray(), SFP_dims, order='F')

maxRawTraces = np.amax(RawTraces)

plt.figure(figsize=(30,15))
plt.subplot(3,2,1); plt.imshow(cn_filter); plt.colorbar(); plt.title('Correlation projection')
plt.subplot(3,2,3); plt.imshow(pnr); plt.colorbar(); plt.title('PNR')
plt.subplot(3,2,5); plt.imshow(np.amax(SFP,axis=2)); plt.colorbar(); plt.title('Spatial footprints')

plt.subplot(3,2,2); plt.figure; plt.title('Example traces (first 20 cells)')
plot_gain = 10 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(RawTraces[i,:],'k')
    else:
      trace = RawTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(RawTraces[i,:],'k')
    else:
      trace = RawTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')

plt.subplot(3,2,4); plt.figure; plt.title('Deconvolved traces (first 20 cells)')
plot_gain = 20 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(DeconvTraces[i,:],'k')
    else:
      trace = DeconvTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(DeconvTraces[i,:],'k')
    else:
      trace = DeconvTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')

#plt.savefig('/tmp/' + 'summary_figure.svg', edgecolor='w', format='svg', transparent=True)

In [ ]:
#cnm.estimates.plot_contours_nb(img=Cn, idx=cnm2.estimates.idx_components)
Cn = cm.local_correlations(images.transpose(1,2,0))
Cn[np.isnan(Cn)] = 0
cnm.estimates.plot_contours_nb(img=Cn, idx=cnm.estimates.idx_components, thr=0.7)

## cnm.estimates.nb_view_components(img=Cn, idx=cnm.estimates.idx_components, thr=0.8)

In [ ]:
# Stop the cluster
cm.stop_server(dview=dview)

In [ ]:
movie = cnm.estimates.play_movie(images, q_max=99.9, gain_res=4,
                                 magnification=2,
                                 bpx=0,
                                 include_bck=False,
                                 use_color=True,
                                 frame_range=range(3000,3500))